In [1]:
%matplotlib inline
import rasterio
import matplotlib.pyplot as plt
import gdal
import numpy as np
import timeit

In [2]:
# Function list
def band_average(raster, bands, icethickness):
    band_average = np.zeros((bands.max()+1, 1))
    for n in range(bands.max()+1):
        raster_single = raster.copy()
        raster_single[bands != n] = np.nan
        raster_single[(icethickness == 0) & (emergence_velocity == 0)] = np.nan
        band_average[n,0] = np.nanmean(raster_single)
    return band_average
    

def emergence_pixels(vel_x_raw, vel_y_raw, vel_total, vel_min, vel_depth_avg_factor, icethickness, bands):
    """ Compute the emergence velocity using an ice flux approach
    """
    # Modify vel_y by multiplying velocity by -1 such that matrix operations agree with flow direction
    #    Specifically, a negative y velocity means the pixel is flowing south.
    #    However, if you were to subtract that value from the rows, it would head north in the matrix.
    #    This is due to the fact that the number of rows start at 0 at the top.
    #    Therefore, multipylying by -1 aligns the matrix operations with the flow direction
    vel_y = vel_y_raw * -1 * vel_depth_avg_factor
    vel_x = vel_x_raw * vel_depth_avg_factor
    # Compute the initial volume
    volume_initial = icethickness * pixel_size**2
    # Quality control options:
    # Apply a border based on the max specified velocity to prevent errors associated with pixels going out of bounds
    if option_border == 1:
        border = int(max_velocity / pixel_size)
        for r in range(vel_x.shape[0]):
            for c in range(vel_x.shape[1]):
                if (r < border) | (r >= vel_x.shape[0] - border) | (c < border) | (c >= vel_x.shape[1] - border):
                    vel_x[r,c] = 0
                    vel_y[r,c] = 0
    # Minimum/maximum velocity bounds
    if option_minvelocity == 1:
        vel_x[vel_total < vel_min] = 0
        vel_y[vel_total < vel_min] = 0
    if option_maxvelocity == 1:
        vel_x[vel_total > max_velocity] = 0
        vel_y[vel_total > max_velocity] = 0
    # Remove clusters of high velocity on stagnant portions of glaciers due to feature tracking of ice cliffs and ponds
    if option_stagnantbands == 1:
        vel_x[bands <= stagnant_band] = 0
        vel_y[bands <= stagnant_band] = 0        
    # Compute displacement in units of pixels
    vel_x_pix = vel_x / pixel_size
    vel_y_pix = vel_y / pixel_size
    # Compute the displacement and fraction of pixels moved for all columns (x-axis)
    # col_x1 is the number of columns to the closest pixel receiving ice [ex. 2.6 returns 2, -2.6 returns -2]
    #    int() automatically rounds towards zero
    col_x1 = vel_x_pix.astype(int)
    # col_x2 is the number of columns to the further pixel receiving ice [ex. 2.6 returns 3, -2.6 returns -3]
    #    np.sign() returns a value of 1 or -1, so it's adding 1 pixel away from zero
    col_x2 = (vel_x_pix + np.sign(vel_x_pix)).astype(int)
    # rem_x2 is the fraction of the pixel that remains in the further pixel (col_x2) [ex. 2.6 returns 0.6, -2.6 returns 0.6]
    #    np.sign() returns a value of 1 or -1, so multiplying by that ensures you have a positive value
    #    then when you take the remainder using "% 1", you obtain the desired fraction
    rem_x2 = np.multiply(np.sign(vel_x_pix), vel_x_pix) % 1
    # rem_x1 is the fraction of the pixel that remains in the closer pixel (col_x1) [ex. 2.6 returns 0.4, -2.6 returns 0.4]
    rem_x1 = 1 - rem_x2
    # Repeat the displacement and fraction computations for all rows (y-axis)
    row_y1 = vel_y_pix.astype(int)
    row_y2 = (vel_y_pix + np.sign(vel_y_pix)).astype(int)
    rem_y2 = np.multiply(np.sign(vel_y_pix), vel_y_pix) % 1
    rem_y1 = 1 - rem_y2
    # Compute the mass flux for each pixel
    volume_final = np.zeros(volume_initial.shape)
    for r in range(vel_x.shape[0]):
        for c in range(vel_x.shape[1]):
            volume_final[r+row_y1[r,c], c+col_x1[r,c]] = (
                volume_final[r+row_y1[r,c], c+col_x1[r,c]] + rem_y1[r,c]*rem_x1[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y2[r,c], c+col_x1[r,c]] = (
                volume_final[r+row_y2[r,c], c+col_x1[r,c]] + rem_y2[r,c]*rem_x1[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y1[r,c], c+col_x2[r,c]] = (
                volume_final[r+row_y1[r,c], c+col_x2[r,c]] + rem_y1[r,c]*rem_x2[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y2[r,c], c+col_x2[r,c]] = (
                volume_final[r+row_y2[r,c], c+col_x2[r,c]] + rem_y2[r,c]*rem_x2[r,c]*volume_initial[r,c]
                )
    # Check that mass is conserved (threshold = 0.1 m x pixel_size**2)
    print('Mass is conserved?', (volume_final.sum() - volume_initial.sum()) < 0.1 * pixel_size**2)
    print(volume_final.sum() - volume_initial.sum())
    # Final ice thickness
    icethickness_final = volume_final / pixel_size**2
    # Emergence velocity
    emergence_velocity = icethickness_final - icethickness
    return emergence_velocity


def export_raster(raster, output_filename, ds, NoData_value):
    driver = gdal.GetDriverByName('GTiff')
    new_dataset = driver.Create(output_filename,
                                ds.RasterXSize,    # number of columns
                                ds.RasterYSize,    # number of rows
                                1,                  # number of bands
                                gdal.GDT_Float32)  # datatype of the raster
    new_dataset.SetProjection(ds.GetProjection())
    new_dataset.SetGeoTransform(ds.GetGeoTransform())
    # Now we need to set the band's nodata value to -1
    new_band = new_dataset.GetRasterBand(1)
    new_band.SetNoDataValue(NoData_value)
    # And finally, let's write our NDVI array.
    new_band.WriteArray(raster)
    

def import_raster(raster_fn):
    """Open raster and obtain the values in its first band as an array
    Output: array of raster values
    """
    # open raster dataset
    raster_ds = gdal.Open(raster_fn)
    # extract band information and get values
    raster_band = raster_ds.GetRasterBand(1)
    raster_values = raster_band.ReadAsArray()
    return raster_values

In [3]:
# # ----- Input for Khumbu Glacier ----------
# # Filenames (fn)
# vel_x_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_20132016_vx_med_ClipV2.tif'
# vel_y_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_20132016_vy_med_ClipV2.tif'
# vel_x_std_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_dshean_vx_std_ClipV2.tif'
# vel_y_std_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_dshean_vy_std_ClipV2.tif'
# icethickness_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_icethickness_Huss.tif'
# bands_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_600m_bands_polygons_raster_ClipV2.tif'
# # Additional input
# mc_simulations = 1000
# pixel_size = 30
# min_velocity_mean = 4.267
# min_velocity_std = 2.799 
# max_velocity = 200
# icethickness_uncertainty_percentage = -21
# #  Farinotti et al. [2017] showed on average Huss and Farinotti [2012] underestimate ice thickness by 21%
# vel_depth_avg_factor_min = 0.8
# vel_depth_avg_factor_max = 0.8
# #  Azam et al. (2012) uses 0.9 (state ice flux is +/- 10% based on unknown sliding and other sources of error)
# #  Nuimura et al. (2011) uses 0.8 based on Paterson (1994) and Sakai et al. (2006)
# #  Vincent et al. (2016) uses 0.8
# stagnant_band = 7

# # ----- Input for Ngozumpa Glacier ----------
# # Filenames
# vel_x_fn = '../DShean_20171025/Ngoz_tiffs/ngozumpa_20112016_vx_clippedV2.tif'
# vel_y_fn = '../DShean_20171025/Ngoz_tiffs/ngozumpa_20112016_vy_clippedV2.tif'
# vel_x_std_fn = '../DShean_20171025/Ngoz_tiffs/Ngoz_dshean_vx_std_clipV2.tif'
# vel_y_std_fn = '../DShean_20171025/Ngoz_tiffs/Ngoz_dshean_vy_std_clipV2.tif'
# icethickness_fn = '../DShean_20171025/Ngoz_tiffs/Ngoz_IceThickness_Huss.tif'
# bands_fn = '../DShean_20171025/Ngoz_tiffs/Ngoz_Bands_600m_ClippedV2.tif'
# # Additional input
# mc_simulations = 1000
# pixel_size = 30
# min_velocity_mean = 5.218
# min_velocity_std = 4.352
# max_velocity = 200
# icethickness_uncertainty_percentage = -21
# vel_depth_avg_factor_min = 0.8
# vel_depth_avg_factor_max = 0.8
# stagnant_band = 11

# ----- Input for Imja-Lhotse Shar Glacier ----------
# Filenames
vel_x_fn = '../DShean_20171025/Imja_tiffs/Imja_dehecq_20132015_vx_clipHuss.tif'
vel_y_fn = '../DShean_20171025/Imja_tiffs/Imja_dehecq_20132015_vy_V2_clipHuss.tif'
#  Note: velocities from Dehecq have positive y moving north to south.  Multiplied by -1 in pre-processing such that they are
#        consistent with DShean velocity maps of negative indicating north to south.
# vel_x_std_fn = '../DShean_20171025/Imja_tiffs/Ngoz_dshean_vx_std_clipV2.tif'
# vel_y_std_fn = '../DShean_20171025/Imja_tiffs/Ngoz_dshean_vy_std_clipV2.tif'
icethickness_fn = '../DShean_20171025/Imja_tiffs/Imja_icethickness_Huss.tif'
bands_fn = '../DShean_20171025/Imja_tiffs/Imja_600m_Bands_polygons_raster_clipHuss.tif'
# Additional input
mc_simulations = 1000
pixel_size = 30
min_velocity_mean = 2
min_velocity_std = 1
max_velocity = 200
icethickness_uncertainty_percentage = -21
vel_depth_avg_factor_min = 0.8
vel_depth_avg_factor_max = 0.8
stagnant_band = 0

# ----- Quality Control Options ----------
# Filter min/max velocity
option_minvelocity = 1
option_maxvelocity = 1
#  > 1 (default) - apply limits specified in model input
#  > 0 - do not apply limits
option_border = 1
#  > 1 (default) - apply the border to prevent errors
#  > 0 - do not apply border, assumes errors near edges are avoided elsewhere
option_stagnantbands = 1
#  > 1 (default) - remove velocities from stagnant bands that are caused by cliffs and ponds
#  > 0 - do not remove the clusters of higher velocities on the tongues of the glaciers

In [4]:
# Open raster for properties (will be used to export tif)
ds = gdal.Open(vel_x_fn)
# Import rasters
vel_x = import_raster(vel_x_fn)
vel_y = import_raster(vel_y_fn)
# Imja estimates using Deheqc, which doesn't have the stdev files
try:
    vel_x_std = import_raster(vel_x_std_fn)
    vel_y_std = import_raster(vel_y_std_fn)
except:
    vel_x_std = np.zeros((vel_x.shape[0], vel_x.shape[1])) + 1.61
    vel_y_std = np.zeros((vel_x.shape[0], vel_x.shape[1])) + 1.61
icethickness = import_raster(icethickness_fn)
bands = import_raster(bands_fn)

In [5]:
# MONTE CARLO SIMULATIONS OF EMERGENCE VELOCITY
# Compute the total velocity
vel_total = (vel_x**2 + vel_y**2)**0.5
#   The total velocity correction, i.e., the minimum velocity based on the total velocity, must be applied prior to the 
#   velocity being adjusted for uncertainties because those uncertainties would change the minimum velocity threshold
#   to the exact same extent.  Hence, the total velocity is computed outside the MC simulations here.
# Generate normal random numbers (mean 0, stdev 1) to include vx and vy uncertainties
mc_vm_uncertainty = np.random.normal(0, 1, (1, mc_simulations))
# Generate normal distribution for minimum velocity threshold
mc_vmin_uncertainty = np.random.normal(min_velocity_mean, min_velocity_std, (1, mc_simulations))
# Generate uniform distribution for ice thickness uncertainty between Huss and -21% underestimation
mc_icethickness_uncertainty = np.random.uniform(icethickness_uncertainty_percentage, 0, (1, mc_simulations))
# Generate uniform distribution for ratio of surface to mean velocity
mc_vsfc_ratio_uncertainty = np.random.uniform(vel_depth_avg_factor_min, vel_depth_avg_factor_max, (1, mc_simulations))
# Run the Monte Carlo simulations
for sim in range(mc_simulations):  
    print('Simulation:', sim)
    # Compute velocities at each pixel according to uncertainty
    mc_vel_x = vel_x + (mc_vm_uncertainty[0, sim] * vel_x_std)
    mc_vel_y = vel_y + (mc_vm_uncertainty[0, sim] * vel_y_std)
    # Minimum velocity including uncertainty
    mc_vmin = mc_vmin_uncertainty[0, sim]
    # Compute ice thicknesses based on uncertainty
    mc_icethickness = icethickness / (1 + mc_icethickness_uncertainty[0, sim]/100)
    #  percent_uncertainty = (x2 - x1) / x1 * 100
    #  percent_uncertainty is the uncertainty (mean = -21%, stdev = 60%)
    #  x2 is the modeled ice thickness (Huss and Farinotti, 2012)
    #  We want to solve for x1, i.e., the ice thickness with the error included
    #    x1 = x2 / (1 + percent_uncertainty / 100)
    mc_vsfc_ratio = mc_vsfc_ratio_uncertainty[0, sim]
    # Emergence Velocity Calculations for each pixel
    emergence_velocity = emergence_pixels(mc_vel_x, mc_vel_y, vel_total, mc_vmin, mc_vsfc_ratio, mc_icethickness, bands)
    # Emergence Velocity Calculations for each band (average the pixel values)
    bands_emergence = band_average(emergence_velocity, bands, icethickness)
    # Record the results
    if sim == 0:
        mc_bands_emergence = bands_emergence
    else:
        mc_bands_emergence = np.concatenate((mc_bands_emergence, bands_emergence), axis=1)
# Calculate statistics of Monte Carlo simulatuions
mc_bands_emergence_wstats = np.zeros((mc_bands_emergence.shape[0],mc_bands_emergence.shape[1]+6))
# Column 01 = band number
bands_col = np.arange(bands.max()+1)
mc_bands_emergence_wstats[:,0] = bands_col
# Column 02 - 06 = median, mean, stdev, min, max
mc_bands_emergence_wstats[:,1] = np.median(mc_bands_emergence, axis=1)
mc_bands_emergence_wstats[:,2] = np.mean(mc_bands_emergence, axis=1)
mc_bands_emergence_wstats[:,3] = np.std(mc_bands_emergence, axis=1)
mc_bands_emergence_wstats[:,4] = np.min(mc_bands_emergence, axis=1)
mc_bands_emergence_wstats[:,5] = np.max(mc_bands_emergence, axis=1)
# Columns 7 - ... = MC simulations
mc_bands_emergence_wstats[:,6:mc_bands_emergence.shape[1]+7] = mc_bands_emergence

Simulation: 0
Mass is conserved? True
18.3029173613
Simulation: 1
Mass is conserved? True
63.0220031738
Simulation: 2
Mass is conserved? False
97.5939788818
Simulation: 3
Mass is conserved? True
-77.6672365665
Simulation: 4
Mass is conserved? True
55.7472990751
Simulation: 5
Mass is conserved? False
120.018722534
Simulation: 6
Mass is conserved? True
-21.3431091309
Simulation: 7
Mass is conserved? True
-1.96473693848
Simulation: 8
Mass is conserved? True
61.2463226318
Simulation: 9
Mass is conserved? True
46.4479522705
Simulation: 10
Mass is conserved? True
88.4528503418
Simulation: 11
Mass is conserved? True
-45.1009368896
Simulation: 12
Mass is conserved? True
-7.03324890137
Simulation: 13
Mass is conserved? True
-80.5257418156
Simulation: 14
Mass is conserved? True
-22.7788848877
Simulation: 15
Mass is conserved? True
-17.2525177002
Simulation: 16
Mass is conserved? True
-25.7147216797
Simulation: 17
Mass is conserved? True
-43.0417633057
Simulation: 18
Mass is conserved? True
-24.0

Mass is conserved? True
40.5114746094
Simulation: 154
Mass is conserved? True
-1.34579479694
Simulation: 155
Mass is conserved? True
-61.7337188721
Simulation: 156
Mass is conserved? False
101.302307129
Simulation: 157
Mass is conserved? True
-13.2912750244
Simulation: 158
Mass is conserved? True
26.2414245605
Simulation: 159
Mass is conserved? True
56.7196807861
Simulation: 160
Mass is conserved? True
-29.0141143799
Simulation: 161
Mass is conserved? True
37.9002685547
Simulation: 162
Mass is conserved? True
71.4261016846
Simulation: 163
Mass is conserved? True
-15.3164672852
Simulation: 164
Mass is conserved? True
44.7618560791
Simulation: 165
Mass is conserved? True
69.1985931396
Simulation: 166
Mass is conserved? True
-31.5618591309
Simulation: 167
Mass is conserved? True
-50.3712003231
Simulation: 168
Mass is conserved? True
4.15867614746
Simulation: 169
Mass is conserved? True
-98.7519834042
Simulation: 170
Mass is conserved? True
-3.64915442467
Simulation: 171
Mass is conserved?

Mass is conserved? True
25.638900876
Simulation: 305
Mass is conserved? True
-47.108581543
Simulation: 306
Mass is conserved? True
-29.8202362061
Simulation: 307
Mass is conserved? True
59.4898375273
Simulation: 308
Mass is conserved? True
81.263885498
Simulation: 309
Mass is conserved? True
-63.2752685547
Simulation: 310
Mass is conserved? True
32.4508821964
Simulation: 311
Mass is conserved? True
81.8503267765
Simulation: 312
Mass is conserved? True
9.99345421791
Simulation: 313
Mass is conserved? True
38.4038391113
Simulation: 314
Mass is conserved? True
50.8522799015
Simulation: 315
Mass is conserved? True
77.2874145508
Simulation: 316
Mass is conserved? True
37.8401031494
Simulation: 317
Mass is conserved? True
76.8258361816
Simulation: 318
Mass is conserved? True
-20.3357543945
Simulation: 319
Mass is conserved? True
10.3431245089
Simulation: 320
Mass is conserved? True
-19.6813201904
Simulation: 321
Mass is conserved? True
76.9073181152
Simulation: 322
Mass is conserved? True
15

Mass is conserved? True
46.9847564697
Simulation: 456
Mass is conserved? True
-17.1938477755
Simulation: 457
Mass is conserved? False
95.7714996338
Simulation: 458
Mass is conserved? True
50.1537780762
Simulation: 459
Mass is conserved? False
110.449996948
Simulation: 460
Mass is conserved? True
66.7970429659
Simulation: 461
Mass is conserved? True
69.2119598389
Simulation: 462
Mass is conserved? True
12.5350341797
Simulation: 463
Mass is conserved? True
38.3126678467
Simulation: 464
Mass is conserved? True
-18.5781097412
Simulation: 465
Mass is conserved? True
87.5600738525
Simulation: 466
Mass is conserved? True
-22.0342254639
Simulation: 467
Mass is conserved? True
-131.237442017
Simulation: 468
Mass is conserved? True
-21.8895876408
Simulation: 469
Mass is conserved? True
-100.520980835
Simulation: 470
Mass is conserved? True
11.2338256836
Simulation: 471
Mass is conserved? True
-39.0241090059
Simulation: 472
Mass is conserved? True
-125.756271362
Simulation: 473
Mass is conserved?

Mass is conserved? True
18.6853027344
Simulation: 607
Mass is conserved? True
42.7308807373
Simulation: 608
Mass is conserved? True
-87.8728485107
Simulation: 609
Mass is conserved? True
7.69340515137
Simulation: 610
Mass is conserved? True
12.7410583496
Simulation: 611
Mass is conserved? True
81.8975222111
Simulation: 612
Mass is conserved? True
-21.4617005587
Simulation: 613
Mass is conserved? True
-65.0243377686
Simulation: 614
Mass is conserved? True
18.8249359131
Simulation: 615
Mass is conserved? True
4.17886340618
Simulation: 616
Mass is conserved? True
-94.9311980009
Simulation: 617
Mass is conserved? True
-4.67073059082
Simulation: 618
Mass is conserved? False
122.523773193
Simulation: 619
Mass is conserved? True
-43.807723999
Simulation: 620
Mass is conserved? True
-47.1723024845
Simulation: 621
Mass is conserved? True
27.7642974854
Simulation: 622
Mass is conserved? True
-32.3610687256
Simulation: 623
Mass is conserved? True
-41.005279541
Simulation: 624
Mass is conserved? F

Mass is conserved? True
8.39936828613
Simulation: 758
Mass is conserved? True
36.3580322266
Simulation: 759
Mass is conserved? True
-9.21578979492
Simulation: 760
Mass is conserved? True
-7.2825012207
Simulation: 761
Mass is conserved? True
-21.6408233643
Simulation: 762
Mass is conserved? True
35.6974945068
Simulation: 763
Mass is conserved? True
78.2891695499
Simulation: 764
Mass is conserved? True
43.1735229492
Simulation: 765
Mass is conserved? False
135.771820068
Simulation: 766
Mass is conserved? True
-81.6645355225
Simulation: 767
Mass is conserved? True
18.8504333496
Simulation: 768
Mass is conserved? True
-47.490524292
Simulation: 769
Mass is conserved? True
69.8241422176
Simulation: 770
Mass is conserved? True
-6.81703186035
Simulation: 771
Mass is conserved? True
-67.9069215059
Simulation: 772
Mass is conserved? True
-36.6513977051
Simulation: 773
Mass is conserved? True
28.4005889893
Simulation: 774
Mass is conserved? True
44.8549802303
Simulation: 775
Mass is conserved? Tr

Mass is conserved? True
49.142288208
Simulation: 909
Mass is conserved? True
30.3916016817
Simulation: 910
Mass is conserved? True
71.4834136963
Simulation: 911
Mass is conserved? True
49.6499786377
Simulation: 912
Mass is conserved? True
-3.24781799316
Simulation: 913
Mass is conserved? True
-60.664291501
Simulation: 914
Mass is conserved? True
59.8207246065
Simulation: 915
Mass is conserved? True
-102.322540283
Simulation: 916
Mass is conserved? True
-44.2272186279
Simulation: 917
Mass is conserved? True
-32.4951933622
Simulation: 918
Mass is conserved? True
67.436416626
Simulation: 919
Mass is conserved? True
88.5135039091
Simulation: 920
Mass is conserved? True
15.109359622
Simulation: 921
Mass is conserved? True
-4.97406005859
Simulation: 922
Mass is conserved? True
58.2445222139
Simulation: 923
Mass is conserved? True
5.37637317181
Simulation: 924
Mass is conserved? True
-99.2571563721
Simulation: 925
Mass is conserved? True
-39.2275085449
Simulation: 926
Mass is conserved? True


In [6]:
np.savetxt('../Output/emergence_Imja_mcstat.txt', mc_bands_emergence_wstats)
# np.savetxt('../Output/emergence_Khumbu_mcstat_stagnantbands.txt', mc_bands_emergence_wstats)
# np.savetxt('../Output/emergence_Khumbu.txt', bands_emergence)

In [7]:
# # Run without uncertainties
# vel_total = (vel_x**2 + vel_y**2)**0.5
# emergence_velocity = emergence_pixels(vel_x, vel_y, vel_total, icethickness)
# bands_emergence = band_average(emergence_velocity, bands, icethickness)
# print(bands_emergence)

In [8]:
# plt.figure(1)
# plt.imshow(vel_x)
# plt.ylabel('row')
# plt.xlabel('col')
# plt.title('x velocity (m a-1)')
# plt.clim(-120, 60)
# plt.colorbar()

# plt.figure(2)
# plt.imshow(vel_y)
# plt.title('y velocity (m a-1)')
# plt.clim(-120, 60)
# plt.colorbar()

# plt.figure(3)
# plt.imshow(icethickness)
# plt.title('ice thickness (m)')
# plt.colorbar()

# vel_total = total_velocity(vel_x, vel_y)
# plt.figure(4)
# plt.imshow(vel_total)
# plt.ylabel('row')
# plt.xlabel('col')
# plt.title('total velocity (m a-1)')
# plt.clim(-120, 60)
# plt.colorbar()